In [9]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
pd.set_option('display.max_columns', 1000)
%matplotlib inline

print("✅ Libraries loaded!")

✅ Libraries loaded!


In [10]:
# Loading the data analyzed in week 1
df = pd.read_csv('../data/accepted_2007_to_2018Q4.csv', low_memory=False)

print(f"✅ Data loaded!")
print(f"Shape: {df.shape}")
print(f"Rows: :{len(df):,}")
print(f"Columns: {len(df.columns)}")

✅ Data loaded!
Shape: (2260701, 151)
Rows: :2,260,701
Columns: 151


In [11]:
# Calculating missing value percentages
missing = (df.isnull().sum() / len(df)*100).sort_values(ascending=False)

# Creating a dataframe for easier viewing
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Percentage': missing.values
})

# Show Only columns with missing values
missing_df = missing_df[missing_df['Missing_Percentage'] >0]

print("Columns with Missing Values:")
print(missing_df.head(20))

Columns with Missing Values:
                                        Column  Missing_Percentage
0                                    member_id          100.000000
1   orig_projected_additional_accrued_interest           99.617331
2                              hardship_reason           99.517097
3               hardship_payoff_balance_amount           99.517097
4                 hardship_last_payment_amount           99.517097
5                      payment_plan_start_date           99.517097
6                                hardship_type           99.517097
7                              hardship_status           99.517097
8                          hardship_start_date           99.517097
9                                deferral_term           99.517097
10                             hardship_amount           99.517097
11                                hardship_dpd           99.517097
12                        hardship_loan_status           99.517097
13                             ha

### Understanding High-Missing Columns

**member_id (100% missing):**
- Removed for privacy protection
- Not useful for modeling anyway
- **Decision:** Drop

**hardship_* columns (99.5 missing):**
- Only populated for loans in hardship programs (rare)
- Examples: hardship_reason, hardship_amount, hardship_status
- These only apply to ~0.5% of loans
- **Decision:** DROP - not enough data to be useful

**settlement_* columns (98.5% missing):**
- Only populated for loans that went to debt settelemtn
- Only ~1.5% of loans
- **Decision:** DROP - insufficient data

**Key Insight:**
These aren't "randomly missing" - they are "structurally missing" (only apply to specific situations).
When >95% missing, the column provides almost no infomration for modeling.

In [12]:
# Applying >50% missing rule
print("="*70)
print("STEP 1: DROPPING COLUMNS WITH >50% MISSING")
print("="*70)

# Calculate missing percentages
missing_pct = (df.isnull().sum() / len(df) * 100)

# Finding columns with >50% missing
high_missing = missing_pct[missing_pct > 50].index.tolist()

print(f"\n Found {len(high_missing)} columns with >50% missing")
print("\nExamples:")
for col in high_missing[:5]:
    print(f" .{col}: {missing_pct[col]:.2f}% missing")
    
# Drop them
df_cleaned = df.drop(columns = high_missing)

print(f"\n ✅ BEFORE: {df.shape[1]} columns")
print(f" ✅ AFTER: {df_cleaned.shape[1]} columns")
print(f" ✅ REMOVED: {len(high_missing)} columns")    

STEP 1: DROPPING COLUMNS WITH >50% MISSING

 Found 44 columns with >50% missing

Examples:
 .member_id: 100.00% missing
 .desc: 94.42% missing
 .mths_since_last_delinq: 51.25% missing
 .mths_since_last_record: 84.11% missing
 .next_pymnt_d: 59.51% missing

 ✅ BEFORE: 151 columns
 ✅ AFTER: 107 columns
 ✅ REMOVED: 44 columns


In [13]:
print("\n" + "="*70)
print("STEP 2: SELECTIVELY DROPPING MEDIUM-HIGH MISSING (>20%)")
print("="*70)

# Recalculate missing values for cleaned dataset
remaining_pct =(df_cleaned.isnull().sum() / len(df_cleaned) * 100)

# FINDING COLUMNS WITH >20% MISSING (BUT ALREADY DROPPED >50%)
medium_high = remaining_pct[(remaining_pct >20) & (remaining_pct <=50)].index.tolist()

# FIND COLUMNS WITH 5-20% MISSING
to_keep = remaining_pct[(remaining_pct >=5) & (remaining_pct <=20)].index.tolist()

print(f"\n Dropping (>20% missing): {len(medium_high)} columns")
print(f"\ Why? Structural missing - not available for old loans")
if len(medium_high) > 0:
    for col in medium_high[:5]:
        print(f" /{col}: {remaining_pct[col]:.2f}%")
              
# Drop only the >20% ones
df_cleaned = df_cleaned.drop(columns=medium_high)
print(f"\n Dropped {len(medium_high)} columns")
print(f"✅ Remaining columns: {df_cleaned.shape[1]}")
             


STEP 2: SELECTIVELY DROPPING MEDIUM-HIGH MISSING (>20%)


<>:15: SyntaxWarning: invalid escape sequence '\ '
<>:15: SyntaxWarning: invalid escape sequence '\ '
C:\Users\zaina\AppData\Local\Temp\ipykernel_35420\3643382890.py:15: SyntaxWarning: invalid escape sequence '\ '
  print(f"\ Why? Structural missing - not available for old loans")



 Dropping (>20% missing): 14 columns
\ Why? Structural missing - not available for old loans
 /open_acc_6m: 38.31%
 /open_act_il: 38.31%
 /open_il_12m: 38.31%
 /open_il_24m: 38.31%
 /mths_since_rcnt_il: 40.25%

 Dropped 14 columns
✅ Remaining columns: 93


In [14]:
# Imputing Critical Features with 5-20% missing (emp_length, emp_title)
print("\n" + "="* 70)
print("STEP 3: IMPUTE CRITICAL FEATURES")
print("="*70)

# Defining Critical Features
critical_features = {
    'annual_inc': ('numerical', 'Income'),
    'dti':('numerical', 'Debt-to-Income Ratio'),
    'revol_util': ('numerical', 'Credit Utilization'),
    'emp_length': ('categorical', 'Employment Length'),
    'emp_title': ('categorical', 'Job Title')
}

for col, (col_type, description) in critical_features.items():
    if col in df_cleaned.columns:
        missing_count = df_cleaned[col].isnull().sum()
        missing_pct = (missing_count / len(df_cleaned)) * 100
        
        if missing_count > 0:
            print(f"\n{'='*60}")
            print(f" {col} - {description}")
            print(f" Missing: {missing_count:,} ({missing_pct:.2f}%)")
            
            # Numerical: Use Median
            if col_type == 'numerical':
                median_value = df_cleaned[col].median()
                print(f" Strategy: Median imputation")
                print(f" Median: {median_value:.2f}")
                print(f" Why? Financial data is skewed (outliers!)")
                
                df_cleaned[col] = df_cleaned[col].fillna(median_value)
                print(f" ✅ Imputed missing values with median")
                
                
            # Categorical
            elif col_type == 'categorical':
                # Creating indicator column
                indicator_name = f"{col}_missing"
                df_cleaned[indicator_name] = df_cleaned[col].isnull().astype(int)
                print(f" Strategy: Mode/Unknown + indicator")
                print(f" ✅ Created '{indicator_name}' flag")   
                
                if col == 'emp_length':
                    # Use mode for emp_length
                    mode_value = df_cleaned[col].mode()[0]
                    print(f" Mode: {mode_value}")
                    df_cleaned[col] = df_cleaned[col].fillna(mode_value)
                    print(f" ✅ Filled with mode")
                else:
                    # Use 'Unknown' for emp_title
                    df_cleaned[col].fillna('Unknown', inplace=True)
                    print(f" ✅ Filled with 'Unknown'")

    # Verifyinng
    remaining = df_cleaned[col].isnull().sum()
    print(f" ✅ Remaining missing: {remaining}")
else: 
    print(f"\n ✅ {col} No missing values!")                         


STEP 3: IMPUTE CRITICAL FEATURES

 annual_inc - Income
 Missing: 37 (0.00%)
 Strategy: Median imputation
 Median: 65000.00
 Why? Financial data is skewed (outliers!)
 ✅ Imputed missing values with median
 ✅ Remaining missing: 0

 dti - Debt-to-Income Ratio
 Missing: 1,744 (0.08%)
 Strategy: Median imputation
 Median: 17.84
 Why? Financial data is skewed (outliers!)
 ✅ Imputed missing values with median
 ✅ Remaining missing: 0

 revol_util - Credit Utilization
 Missing: 1,835 (0.08%)
 Strategy: Median imputation
 Median: 50.30
 Why? Financial data is skewed (outliers!)
 ✅ Imputed missing values with median
 ✅ Remaining missing: 0

 emp_length - Employment Length
 Missing: 146,940 (6.50%)
 Strategy: Mode/Unknown + indicator
 ✅ Created 'emp_length_missing' flag
 Mode: 10+ years
 ✅ Filled with mode
 ✅ Remaining missing: 0

 emp_title - Job Title
 Missing: 167,002 (7.39%)
 Strategy: Mode/Unknown + indicator
 ✅ Created 'emp_title_missing' flag


C:\Users\zaina\AppData\Local\Temp\ipykernel_35420\226235424.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna('Unknown', inplace=True)


 ✅ Filled with 'Unknown'
 ✅ Remaining missing: 0

 ✅ emp_title No missing values!


In [15]:
print("\n" + "="*70)
print("STEP 4: HANDEL REMAINING LOW-MISSING COLUMNS")
print("="*70)

# Check what's left
remaining_pct = (df_cleaned.isnull().sum() / len(df_cleaned) * 100)
low_missing = remaining_pct[(remaining_pct > 0) & (remaining_pct <5)].sort_values(ascending=False)

if len(low_missing) > 0:
    print(f"\n Found {len(low_missing)} columns with <5% missing:\n")
    print(low_missing.head(10))
    
    print("\n 💡 Imputing with median (numerical) or mode (categorical)...")
    
    for col in low_missing.index:
        # Check data type
        if df_cleaned[col].dtype in ['float64', 'int64']:
            # Numerical - median
            median_value = df_cleaned[col].median()
            df_cleaned[col] = df_cleaned[col].fillna(median_value)
        else:
            # Categorical - mode or 'Unknown'
            if df_cleaned[col].mode().shape[0] > 0:
                mode_value = df_cleaned[col].mode()[0]
                df_cleaned[col] = df_cleaned[col].fillna(mode_value)
            else:
                df_cleaned[col] = df_cleaned[col].fillna('Unknown')
                
    print(f" \n ✅ Imputed {len(low_missing)} columns")
else:
    print("\n ✅ No remaining low-missing columns!")


STEP 4: HANDEL REMAINING LOW-MISSING COLUMNS

 Found 84 columns with <5% missing:

bc_util                  3.366389
percent_bc_gt_75         3.335779
bc_open_to_buy           3.316140
mths_since_recent_bc     3.248771
pct_tl_nvr_dlq           3.116909
avg_cur_bal              3.113149
num_rev_accts            3.110097
mo_sin_old_rev_tl_op     3.110097
mo_sin_rcnt_rev_tl_op    3.110097
tot_coll_amt             3.110053
dtype: float64

 💡 Imputing with median (numerical) or mode (categorical)...
 
 ✅ Imputed 84 columns


In [16]:
print("\n" + "="*70)
print("STEP 4.5: HANDEL SPECIAL 'MONTHS SINCE' COLUMNS")
print("="*70)

# These columns: Missing = "Never Happened"
# We fill with a large number (999) to indicate "never occurred"

special_cols ={ 
               'mths_since_recent_inq': 'Months isnce recent inquiry',
               'num_tl_120dpd_2m': 'Severly delinquent accounts',
               'mo_sin_old_il_acct': 'Months since oldest installment loan',
               'mths_since_last_delinq':'Months since last delinquency',
               'mths_since_last_record': 'Months since last public record',
                'mths_since_last_major_derog': 'Months since last major derogatory'
               }

print("\n Strategy: Fill with 999 (indicates 'never occurred')")
print(" Why? Missing = 'never had this problem' = GOOD for borrower!\n")

for col, description in special_cols.items():
    if col in df_cleaned.columns:
        missing_count = df_cleaned[col].isnull().sum()
        
        if missing_count> 0:
            missing_pct = (missing_count / len(df_cleaned)) * 100
            print(f" {col}")
            print(f" Description: {description}")
            print(f" Missing: {missing_count:,} ({missing_pct:.2f}%)")

        if col.startswith('num_'):
            df_cleaned[col] = df_cleaned[col].fillna(0)
            print(f" ✅ Filled with 0 (never had this)\n")
        else:
            # For "months since", fill with 999
            df_cleaned[col] = df_cleaned[col].fillna(999)
            print(f" ✅ Filled with 999 (never occurred)")
            
            
print("="* 60)
print(" ✅ Special columns handeled with domain knowledge!")
print("="*60)


STEP 4.5: HANDEL SPECIAL 'MONTHS SINCE' COLUMNS

 Strategy: Fill with 999 (indicates 'never occurred')
 Why? Missing = 'never had this problem' = GOOD for borrower!

 mths_since_recent_inq
 Description: Months isnce recent inquiry
 Missing: 295,468 (13.07%)
 ✅ Filled with 999 (never occurred)
 num_tl_120dpd_2m
 Description: Severly delinquent accounts
 Missing: 153,690 (6.80%)
 ✅ Filled with 0 (never had this)

 mo_sin_old_il_acct
 Description: Months since oldest installment loan
 Missing: 139,104 (6.15%)
 ✅ Filled with 999 (never occurred)
 ✅ Special columns handeled with domain knowledge!


In [17]:
print("="*70)
print("FINAL FIX AND VERIFICATION")
print("="*70)

# Step 1: Fix the 3 remaining columns
print(f"\n📊 BEFORE fix: {df_cleaned.isnull().sum().sum():,} missing values")

special_cols = {
    'mths_since_recent_inq': 999,  # Never had inquiry
    'num_tl_120dpd_2m': 0,         # Never severely delinquent
    'mo_sin_old_il_acct': 999      # Never had installment loan
}

for col, fill_value in special_cols.items():
    if col in df_cleaned.columns:
        before = df_cleaned[col].isnull().sum()
        if before > 0:
            df_cleaned[col] = df_cleaned[col].fillna(fill_value)
            print(f"  ✅ {col}: Filled {before:,} values with {fill_value}")

# Step 2: Verify zero missing
print(f"\n📊 AFTER fix: {df_cleaned.isnull().sum().sum():,} missing values")

total_missing = df_cleaned.isnull().sum().sum()

if total_missing == 0:
    print("\n" + "="*70)
    print("🎉🎉🎉 SUCCESS! ZERO MISSING VALUES! 🎉🎉🎉")
    print("="*70)
    
    # Step 3: Save the file
    import os
    output_path = '../data/cleaned_accepted_loans.csv'
    print(f"\n💾 Saving to: {output_path}")
    
    df_cleaned.to_csv(output_path, index=False)
    
    size_mb = os.path.getsize(output_path) / 1024**2
    print(f"✅ Saved successfully!")
    print(f"✅ Size: {size_mb:.2f} MB")
    print(f"✅ Rows: {len(df_cleaned):,}")
    print(f"✅ Columns: {df_cleaned.shape[1]}")
    print(f"✅ Missing: {df_cleaned.isnull().sum().sum():,}")
    
    print("\n" + "="*70)
    print("🎉 DAY 1 COMPLETE! READY FOR DAY 2! 🎉")
    print("="*70)
else:
    print(f"\n⚠️  ERROR: Still have {total_missing:,} missing values")
    remaining = df_cleaned.isnull().sum()
    print(remaining[remaining > 0])

FINAL FIX AND VERIFICATION

📊 BEFORE fix: 0 missing values

📊 AFTER fix: 0 missing values

🎉🎉🎉 SUCCESS! ZERO MISSING VALUES! 🎉🎉🎉

💾 Saving to: ../data/cleaned_accepted_loans.csv
✅ Saved successfully!
✅ Size: 1370.65 MB
✅ Rows: 2,260,701
✅ Columns: 95
✅ Missing: 0

🎉 DAY 1 COMPLETE! READY FOR DAY 2! 🎉


In [18]:
print("\n" + "="*70)
print("FINAL VERIFICATION - DAY 1 COMPLETE!")
print("="*70)

# Check for any remaining missing values
total_missing = df_cleaned.isnull().sum().sum()

print(f"\nTotal missing values: {total_missing:,}")

if total_missing == 0:
    print("\n 🎉🎉🎉 SUCCESS! ZERO MISSING VALUES! 🎉🎉🎉")
    print("\n ✅ Dataset is READY for modeling!")
else:
    remaining = df_cleaned.isnull().sum()
    remaining = remaining[remaining >0].sort_values(ascending=False)
    print(remaining.head(10))
    
print(f"\n{'='*60}")
print("FINAL DATASET SUMMARY")
print(f"{'='*60}")
print(f"Rows: {len(df_cleaned):,}")
print(f"Columns: {df_cleaned.shape[1]}")
print(f"Missing values: {total_missing:,}")

# Checking critical features
critical_cols = ['annual_inc', 'dti', 'revol_util', 'emp_length', 'emp_title']
present = [col for col in critical_cols if col in df_cleaned.columns]

print(f"\n ✅ Critical features present: {len(present)}/{len(critical_cols)}")
for col in present:
    print(f" . {col}")

# Check for new indicator columns
indicator_cols = [col for col in df_cleaned.columns if '_missing' in col]
print(f"\n Indicator columns created: {len(indicator_cols)}")
for col in indicator_cols:
    print(f" .{col}")
    
print("\n" + "="*70)
print("DAY 1: MISSING VALUE HANDELING - COMPLETED! 🎉") 
print("="*70)   


FINAL VERIFICATION - DAY 1 COMPLETE!

Total missing values: 0

 🎉🎉🎉 SUCCESS! ZERO MISSING VALUES! 🎉🎉🎉

 ✅ Dataset is READY for modeling!

FINAL DATASET SUMMARY
Rows: 2,260,701
Columns: 95
Missing values: 0

 ✅ Critical features present: 5/5
 . annual_inc
 . dti
 . revol_util
 . emp_length
 . emp_title

 Indicator columns created: 2
 .emp_length_missing
 .emp_title_missing

DAY 1: MISSING VALUE HANDELING - COMPLETED! 🎉


In [19]:
import os

print(f"df_cleaned shape: {df_cleaned.shape}")
print(f" Missing values: {df_cleaned.isnull().sum().sum()}")

output_path ='../data/cleaned_accepted_loans.csv'
print(f"\n Saving to: {output_path}")

df_cleaned.to_csv(output_path, index=False)

if os.path.exists(output_path):
    size_mb =os.path.getsize(output_path) / 1024**2
    print(f"✅ File saved successfully!")
    print(f"✅ Size: {size_mb:.2f} MB")
    print(f"✅ Location: {os.path.abspath(output_path)}")
else:
    print("❌ Save failed!")

df_cleaned shape: (2260701, 95)
 Missing values: 0

 Saving to: ../data/cleaned_accepted_loans.csv
✅ File saved successfully!
✅ Size: 1370.65 MB
✅ Location: c:\Projects\Credit-Risk-Prediction-ML\data\cleaned_accepted_loans.csv
